In [1]:
import sys
import os
os.chdir('..')
sys.path.insert(0, os.getcwd())
import numpy as np
from Data_loader import load_subjects_from_json, get_all_npy_paths_by_group, base_folders

import math
import torch
from time import time
from torch.utils.data import Dataset, DataLoader


In [2]:
from AE_pipeline_pytorch import (
    LSTMAutoencoder,
    BiLSTMAutoencoder,
    create_dataloader,
    create_test_dataloader,
    train_autoencoder,
    evaluate_and_detect,
    extract_and_save_latents,
    reconstruct_and_evaluate 
)

2025-06-26 12:03:58.698350: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750950238.715268  187306 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750950238.720635  187306 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750950238.734322  187306 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750950238.734345  187306 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750950238.734347  187306 computation_placer.cc:177] computation placer alr

Using device: cuda


In [3]:
BATCH_SIZE = 256
NUM_BIOMECHANICAL_VARIABLES = 321
n_timesteps= 100 #cycle is normalized to 100 points 

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [5]:
#Load subjects lists 
train_subjects = {
    "G01": load_subjects_from_json("G01_train_subjects.json"),
    "G03": load_subjects_from_json("G03_train_subjects.json")
}
val_subjects = {
    "G01": load_subjects_from_json("G01_validation_subjects.json"),
    "G03": load_subjects_from_json("G03_validation_subjects.json")
}
test_subjects = {
    "G01": load_subjects_from_json("G01_test_subjects.json"),
    "G03": load_subjects_from_json("G03_test_subjects.json")
}

In [6]:
# Generate routes .npy
train_npy = get_all_npy_paths_by_group(train_subjects, base_folders)
val_npy   = get_all_npy_paths_by_group(val_subjects,   base_folders)
test_npy  = get_all_npy_paths_by_group(test_subjects,  base_folders)

print(f"Train .npy: {len(train_npy)} files")
print(f" Val  .npy: {len(val_npy)} files")
print(f" Test .npy: {len(test_npy)} files")

Train .npy: 932 files
 Val  .npy: 196 files
 Test .npy: 169 files


In [7]:
# Data loaders
# 1) DataLoader para train y validation (sin metadata)
train_loader = create_dataloader(
        train_npy,
        batch_size=BATCH_SIZE,
        is_train=True,
        n_timesteps=n_timesteps,
        n_vars=NUM_BIOMECHANICAL_VARIABLES
    )
val_loader = create_dataloader(
        val_npy,
        batch_size=BATCH_SIZE,
        is_train=False,
        n_timesteps=n_timesteps,
        n_vars=NUM_BIOMECHANICAL_VARIABLES
    )

# 2) DataLoader para test (con metadata)
test_loader = create_test_dataloader(
        test_npy,
        subjects_by_group=test_subjects,
        base_folders=base_folders,
        batch_size=BATCH_SIZE,
        n_timesteps=n_timesteps,
        n_vars=NUM_BIOMECHANICAL_VARIABLES
    )


In [8]:
#Train 

model = LSTMAutoencoder(n_timesteps=100, n_vars=321, latent_dim=128).to(device)


# Compila el modelo (PyTorch 2.0 JIT), **antes** de entrenar
if hasattr(torch, "compile"):
    model = torch.compile(model)


# Tu función de train_autoencoder acepta train_loader y val_loader
train_autoencoder(
    model,
    train_loader,
    val_loader,
    run_id="Torch1",
    epochs=20
)

/home/dmartinez/Gait-Stability/AE_pipeline_pytorch.py:209: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/home/dmartinez/Gait-Stability/AE_pipeline_pytorch.py:222: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/home/dmartinez/Gait-Stability/AE_pipeline_pytorch.py:222: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


RuntimeError: Caught RuntimeError in pin memory thread for device 0.
Original Traceback (most recent call last):
  File "/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 43, in do_one_step
    data = pin_memory(data, device)
  File "/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 100, in pin_memory
    clone[i] = pin_memory(item, device)
  File "/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 66, in pin_memory
    return data.pin_memory(device)
RuntimeError: CUDA error: invalid argument
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



In [ ]:
model.eval()
mse = torch.nn.MSELoss(reduction='none')

all_scores = []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        recon = model(x)                       # (B, T, 321)
        # Si quieres comparar recon vs y, quizá tu función `evaluate_and_detect`
        # espera (x, y) y devuelve errores. Sino:
        # ejemplo de score: media de MSE por variable
        loss = mse(recon, x).mean(dim=(1,2))   # reconstrucción de x vs x
        # o si tu supervisión es recon vs y:
        # loss = mse(recon, y).mean(dim=(1,2))
        all_scores.append(loss.cpu())

all_scores = torch.cat(all_scores)
print("Test MSE (batch-wise):", all_scores.mean().item())
